# Transfer Learning with Tensorflow part 2: Fine-Tuning

In [1]:
# Check if we're using GPU
!nvidia-smi


Sat Jan 13 14:45:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Creating helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-01-13 14:45:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-13 14:45:52 (71.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import *

## Load the data

In [4]:
# get 10% of training data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2024-01-13 14:45:58--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.172.207, 142.251.171.207, 173.194.206.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.172.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   117MB/s    in 1.4s    

2024-01-13 14:46:00 (117 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
unzip_data('10_food_classes_10_percent.zip')

In [6]:
# Check out how many images and subdirectories are in our dataset
walk_through_dir('10_food_classes_10_percent/')

There are 2 directories and 0 images in '10_food_classes_10_percent/'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/g

In [7]:
# Create training and test directory paths
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [8]:
import tensorflow as tf

IMG_SIZE = (224,224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode='categorical',
                                                                            batch_size=BATCH_SIZE,
                                                                            shuffle=True)

test_data= tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                               image_size=IMG_SIZE,
                                                               label_mode='categorical',
                                                               batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [9]:
# Check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [10]:
for images, labels in train_data_10_percent.take(1):
  print(images, labels)

tf.Tensor(
[[[[2.45571426e+02 2.55000000e+02 2.48142853e+02]
   [2.47234695e+02 2.55000000e+02 2.47545914e+02]
   [2.52127548e+02 2.54556122e+02 2.52566330e+02]
   ...
   [2.44841888e+02 1.95841888e+02 1.17270409e+02]
   [2.47714294e+02 1.99760223e+02 1.17760223e+02]
   [2.49387772e+02 2.02030624e+02 1.20030624e+02]]

  [[2.37260208e+02 2.54239807e+02 2.54857147e+02]
   [2.44571442e+02 2.54857147e+02 2.54724487e+02]
   [2.47459183e+02 2.54857147e+02 2.51489792e+02]
   ...
   [2.33046021e+02 1.83403229e+02 1.05046021e+02]
   [2.42867340e+02 1.97933701e+02 1.14933685e+02]
   [2.44928635e+02 2.03377686e+02 1.19117447e+02]]

  [[2.44005112e+02 2.51076538e+02 2.50362244e+02]
   [2.44698990e+02 2.52270416e+02 2.41127548e+02]
   [2.51744904e+02 2.54908157e+02 2.32392853e+02]
   ...
   [2.40693863e+02 1.93785736e+02 1.13739799e+02]
   [2.37142822e+02 1.95469376e+02 1.11841812e+02]
   [2.41076813e+02 2.03076813e+02 1.18505386e+02]]

  ...

  [[2.02658035e+02 2.16658035e+02 2.17658035e+02]
   [2

In [11]:
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=IMG_SIZE+(3,))
base_model.trainable = False

inputs=tf.keras.Input(shape=IMG_SIZE+(3,))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, output)

16705208/16705208 [==============================] - 0s 0us/step


In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [13]:
model.fit(train_data_10_percent,
          epochs=5,
          validation_data=test_data,
          steps_per_epoch=len(train_data_10_percent),
          validation_steps=len(test_data))

Epoch 1/5
24/24 [==============================] - 21s 446ms/step - loss: 1.8559 - accuracy: 0.4253 - val_loss: 1.2810 - val_accuracy: 0.7496
Epoch 2/5
24/24 [==============================] - 7s 277ms/step - loss: 1.0462 - accuracy: 0.7867 - val_loss: 0.8573 - val_accuracy: 0.8196
Epoch 3/5
24/24 [==============================] - 7s 280ms/step - loss: 0.7450 - accuracy: 0.8467 - val_loss: 0.6943 - val_accuracy: 0.8444
Epoch 4/5
24/24 [==============================] - 8s 321ms/step - loss: 0.5998 - accuracy: 0.8773 - val_loss: 0.6052 - val_accuracy: 0.8588
Epoch 5/5
24/24 [==============================] - 8s 342ms/step - loss: 0.5073 - accuracy: 0.9013 - val_loss: 0.5566 - val_accuracy: 0.8620


## Model 0: Building a transfer learning model using the keras Functional API

In [ ]:
# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2.Freeze the base model
base_model.trainable = False

# 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')

# 4. if using ResNet50V2 you will need to normalize inputs
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)(inputs)

# 5. pass the inputs to the base_model
x = base_model(inputs)
print(f'shape after passing inputs through base model:{x.shape}')

# 6. Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f'shape after GlabalAveragePooling2D: {x.shape}')

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. compile the model
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

# 10. fit the model
history_0 = model_0.fit(train_data_10_percent,
                        epochs=5,
                        steps_per_epoch=len(train_data_10_percent),
                        validation_data=test_data,
                        validation_steps=len(test_data))

shape after passing inputs through base model:(None, 7, 7, 1280)
shape after GlabalAveragePooling2D: (None, 1280)
Epoch 1/5
24/24 [==============================] - 22s 583ms/step - loss: 1.9193 - accuracy: 0.4107 - val_loss: 1.3374 - val_accuracy: 0.7384
Epoch 2/5
23/24 [===========================>..] - ETA: 0s - loss: 1.1065 - accuracy: 0.7731